In [ ]:
import pandas as pd
import page_parsing
import json
import re

import importlib
importlib.reload(page_parsing)

prods_urls={
    'laptop': ('laptop', 'https://fptshop.com.vn/may-doi-tra/may-tinh-xach-tay-cu-gia-re', 'https://fptshop.com.vn/may-tinh-xach-tay'),
    'phone': ('phone', 'https://fptshop.com.vn/may-doi-tra/dien-thoai-cu-gia-re', 'https://fptshop.com.vn/dien-thoai'),
    'tablet': ('tablet', 'https://fptshop.com.vn/may-doi-tra/may-tinh-bang-cu-gia-re', 'https://fptshop.com.vn/may-tinh-bang')
}
filename, old_prods_url, new_prods_url=prods_urls['tablet'] #option
filename='data/'+filename

In [ ]:
def new_prods_list():
    prods=page_parsing.get_products_overview(old_prods_url)
    with open(filename+'.json', 'w') as f:
        json.dump(prods, f)
new_prods_list()

In [ ]:
with open(filename+'.json', 'r') as f:
    prods=json.load(f)

for i in range(len(prods)):
    if len(prods[i])>4: #got detail
        continue

    prod_link=prods[i]['Link']
    prod_link=new_prods_url+'/'+re.findall(r'\/([^\/]*)-htm', prod_link)[0]
    try: 
        row=page_parsing.get_product_detail(prod_link)
    except page_parsing.NoSuchElementException:
        print('!!! Cant scrape:', prod_link)
    else:
        prods[i].update(row)
        print('+1 detail:', prod_link)
        with open(filename+'.json', 'w') as f:
            json.dump(prods, f)
        print('saved')


In [ ]:
def save_to_excel():
    with open(filename+'.json', 'r') as f:
        data=json.load(f)

    data_df=pd.DataFrame(data)
    data_df.set_index(data_df.columns[0], inplace=True)
    data_df.to_excel(filename+'.xlsx')
save_to_excel()